In [1]:
# What version of Python do you have?
import sys
import platform
import torch
import pandas as pd
import sklearn as sk

has_gpu = torch.cuda.is_available()
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("NVIDIA/CUDA GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")

Python Platform: macOS-14.3.1-arm64-arm-64bit
PyTorch Version: 2.2.1

Python 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
Pandas 2.2.1
Scikit-Learn 1.4.1.post1
NVIDIA/CUDA GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


In [105]:
import torch
import torch.nn as nn
import torch.optim as optim

class ToffoliApproximator(nn.Module):
    def __init__(self):
        super(ToffoliApproximator, self).__init__()
        # Initialize parameters for 5 2-qubit gates
        self.parameters = nn.ParameterList([nn.Parameter(torch.randn(4, 4, 2, requires_grad=True)) for _ in range(5)])
        self.ccx = torch.tensor(
            [[1., 0., 0., 0., 0., 0., 0., 0.],
            [0., 1., 0., 0., 0., 0., 0., 0.],
            [0., 0., 1., 0., 0., 0., 0., 0.],
            [0., 0., 0., 1., 0., 0., 0., 0.],
            [0., 0., 0., 0., 1., 0., 0., 0.],
            [0., 0., 0., 0., 0., 1., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1.],
            [0., 0., 0., 0., 0., 0., 1., 0.]]
        ).to(torch.complex64)
    
    def forward(self):
        # Convert parameters to complex matrices and construct the circuit
        matrices = [torch.view_as_complex(param) for param in self.parameters()]
        M1 = matrices[0]
        M2 = matrices[1]
        M3 = matrices[2]
        M4 = matrices[3]
        M5 = matrices[4]

        # Implement circuit logic here
        # This involves matrix multiplication and ensuring the gates are applied correctly
        
        # U1 = I tensor_product M1
        U1 = torch.kron(torch.eye(2), matrices[0])

        # U2 = M2 tensor_product I
        U2 = torch.kron(matrices[1], torch.eye(2))

        # U3 = I tensor_product M3
        U3 = torch.kron(torch.eye(2), matrices[2])

        # U4 = M4 tensor_product I
        U4 = torch.kron(matrices[3], torch.eye(2))

        # U5 = I tensor_product M5
        U5 = torch.kron(torch.eye(2), matrices[4])

        print(U1)
        print(U2)
        print(U3)
        print(U4)
        print(U5)

        # W = U1U2U3U4U5
        W = U1 @ U2 @ U3 @ U4 @ U5
        W1 = U1 @ U2
        W2 = U3 @ U4
        print(f"W1: {W1}")
        print(f"W2: {W2}")
        
        W3 = W1 @ W2 

        print(f"W3: {W3}")

        W = W3 @ U5

        # print(W)

        # V = (I⊗I⊗H)W(I⊗I⊗H)
        H = 1 / torch.sqrt(torch.tensor(2.)) * torch.tensor([[1, 1], [1, -1]])
        I = torch.eye(2)
        # V is a complex matrix
        V = torch.kron(torch.kron(I, I), H).to(W.dtype) @ W @ torch.kron(torch.kron(I, I), H).to(W.dtype)
        # Return the output of the circuit
        return V
    
    def unitarity_loss(self):
        loss = 0
        params = list(self.parameters())
        for i in range(1, 5):  # Skip the first matrix since the regularization starts from the second
            matrix = torch.view_as_complex(params[i])
            loss += torch.norm(matrix @ matrix.conj().transpose(-2, -1) - torch.eye(4), 'fro')
        return loss

    def hilbert_schmidt_distance(self, output, target):
        # Implement calculation of Hilbert-Schmidt distance
        
        # First we calculate Tr(CC(X)† V)
        ccx = self.ccx
        ccx_dagger = ccx.conj().transpose(-2, -1)
        # print(f"ccx_dagger: {ccx_dagger}")
        # print(f"output: {output}")
        ccx_dagger_V = ccx_dagger @ output
        tr_ccx_dagger_V = torch.trace(ccx_dagger_V)
        # print(f"(hilbert_schmidt) {torch.abs(tr_ccx_dagger_V)}")
        # d(V,CC(X) = 1 - (|Tr(CC(X)† V)|^2) / (2^3)^2 + e) 
        # d = torch.sqrt(1 - (torch.abs(tr_ccx_dagger_V) ** 2) / 64)
        print(f"tr_ccx_dagger_V: {tr_ccx_dagger_V}")
        print(f"tr_ccx_dagger_V.abs(): {torch.abs(tr_ccx_dagger_V)}")
        d = 1 - (torch.abs(tr_ccx_dagger_V) ** 2) / 64
        return d
    


In [106]:
model = ToffoliApproximator()
# model.forward()
# model.ccx
# print(model.unitarity_loss())

res = model.forward()
res2 = model.ccx

print(model.hilbert_schmidt_distance(model.forward(), model.ccx))

tensor([[-2.6627-0.5597j, -0.1197-1.1330j,  0.4456+1.2785j,  0.3678+0.2011j,
          0.0000-0.0000j,  0.0000-0.0000j,  0.0000+0.0000j,  0.0000+0.0000j],
        [ 0.7530+0.4300j, -1.6707-1.7368j,  0.3175+1.2352j,  1.9960+0.2461j,
          0.0000+0.0000j,  0.0000-0.0000j,  0.0000+0.0000j,  0.0000+0.0000j],
        [ 0.7693+0.5887j,  0.3434+1.1532j,  2.1366+0.0597j, -2.3478+0.1795j,
          0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j, -0.0000+0.0000j],
        [ 0.2122+0.9803j,  1.0421+1.0830j,  0.1336-0.2269j, -0.0250-1.0999j,
          0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000-0.0000j],
        [ 0.0000-0.0000j,  0.0000-0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
         -2.6627-0.5597j, -0.1197-1.1330j,  0.4456+1.2785j,  0.3678+0.2011j],
        [ 0.0000+0.0000j,  0.0000-0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.7530+0.4300j, -1.6707-1.7368j,  0.3175+1.2352j,  1.9960+0.2461j],
        [ 0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j, -0.0000+0.

In [101]:
print(res)
print(res2)

tensor([[ -8.2354-1.9561e+01j,  -8.2383-3.1597e+00j,  -8.1740-5.3461e+00j,
           1.2416+5.1875e-02j,  -6.0905-4.5686e+01j, -13.8559-8.9960e+00j,
          25.4847-8.4386e+00j,   1.7472-3.9008e+00j],
        [-42.6181-5.8550e+01j,  -0.2744-4.1658e+01j,  -6.2540+3.1592e+00j,
          32.2808-1.2768e+01j, -19.8785+6.3062e-01j,   9.1808+3.6511e+01j,
         -13.1274-1.0902e+01j,   1.4061+2.5715e+01j],
        [-39.1279-5.6895e+01j, -20.4133-2.2072e+01j,   9.1896-2.3880e+01j,
          12.7016-1.1839e+01j,  13.2375+2.5080e+01j,  22.7869+5.5532e+00j,
           0.5205-6.7585e+00j,   7.9256+1.0158e+01j],
        [ 31.3767+6.4651e+01j,  -3.6081+1.3484e+01j,  22.2574+2.3575e+01j,
         -22.1919-6.1281e+00j,  -9.7293+4.4460e+01j,  17.2560-1.2541e+01j,
         -19.0475+2.0256e+01j,   3.1432-1.1895e+01j],
        [-16.7646+1.9895e+01j,  -6.2570+6.6263e+00j,  -5.3008-1.8010e+01j,
          -8.3378-3.1513e+00j,  45.5096+3.0540e+01j, -23.3369-4.3429e+01j,
          -8.7314+1.4966e+01j, -19

In [104]:
# Initialize model, optimizer, etc.
model = ToffoliApproximator()
optimizer = optim.Adam(model.parameters(), lr=0.001)
target_matrix = model.ccx

epochs = 10
min_distance = float('inf')

# Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model()
    distance_loss = model.hilbert_schmidt_distance(output, target_matrix)
    unitarity_loss = model.unitarity_loss()
    loss = distance_loss + 1e8 * unitarity_loss
    loss.backward()
    optimizer.step()

    if distance_loss < min_distance:  # Step 2: Update the minimum distance if the current distance is lower
        min_distance = distance_loss.item()  # Convert to a Python float for reporting
    
    print(f"Epoch {epoch+1}, Hilbert-Schmidt Distance: {distance_loss.item()}")

print(f"Minimum Hilbert-Schmidt Distance Achieved: {min_distance}")

tr_ccx_dagger_V: (143.89842224121094-49.278934478759766j)
tr_ccx_dagger_V.abs(): 152.1024932861328
Epoch 1, Hilbert-Schmidt Distance: -360.48699951171875
tr_ccx_dagger_V: (143.92843627929688-49.090206146240234j)
tr_ccx_dagger_V.abs(): 152.0698699951172
Epoch 2, Hilbert-Schmidt Distance: -360.33197021484375
tr_ccx_dagger_V: (143.9568328857422-48.901954650878906j)
tr_ccx_dagger_V.abs(): 152.0360870361328
Epoch 3, Hilbert-Schmidt Distance: -360.17144775390625
tr_ccx_dagger_V: (143.9835205078125-48.71429443359375j)
tr_ccx_dagger_V.abs(): 152.00111389160156
Epoch 4, Hilbert-Schmidt Distance: -360.0052795410156
tr_ccx_dagger_V: (144.00852966308594-48.52727127075195j)
tr_ccx_dagger_V.abs(): 151.9649658203125
Epoch 5, Hilbert-Schmidt Distance: -359.8336181640625
tr_ccx_dagger_V: (144.0317840576172-48.34088897705078j)
tr_ccx_dagger_V.abs(): 151.92759704589844
Epoch 6, Hilbert-Schmidt Distance: -359.6561584472656
tr_ccx_dagger_V: (144.0532684326172-48.155338287353516j)
tr_ccx_dagger_V.abs(): 151